In [1]:
import os
import json

from react_agent.src.config.system_parameters import CodebaseSearcherSettings

from react_agent.src.agent_tools.source_code_retriever import SourceCodeRetriever

from react_agent.src.util.abap_repository import ABAPClassRepository
from react_agent.src.util.memory_manager import MemoryManager
from react_agent.src.util.code_summarizer import CodeSummarizer

In [2]:
notebook_dir = os.path.abspath("")

file_path = os.path.abspath(
    os.path.join(notebook_dir, "./", "resources", "all_classes.json")
)

with open(file_path, encoding="utf8") as f:
    json_file = json.load(f)

In [3]:
classes: list[tuple] = []

for my_class in json_file["classes"]:
    classes.append((my_class["name"].lower(), my_class["description"]))

for my_class in classes:
    print(f"Class name: {my_class[0]}, Description: {my_class[1]}")

Class name: cl_edoc_sg_apj_jt_check, Description: EDOC SG Application Job Catalog Entries Check Class
Class name: cl_edoc_map_escan_fica, Description: CI/FICA SII eDoc map for Canary Islands
Class name: cl_ficaessii_utility, Description: Utility class for ES SII Edocs
Class name: cl_edoc_reconcile_cl, Description: Chile Reconciliation
Class name: cl_edoc_map_ro_ci, Description: eDocument Mapping Romania for CI
Class name: cl_edoc_cisa_payment_means, Description: Saudi Arabia eDocument Payment Means XML Tag
Class name: cl_edoc_joule_error_hndlg, Description: Class for BAdI Impl.: EDOC_ERROR_HANDLING
Class name: cl_edc_result_list_in_gstr, Description: eDocument Consistency India Result Processing
Class name: cl_edocument_sa, Description: eDocument Saudi Arabia
Class name: cl_edoc_map_sa_fi, Description: eDocument Mapping SA FI
Class name: cl_edc_convaltyp_es_sii_taxdrc, Description: eDocument Consistency - Check Engine Logic
Class name: cl_edoc_intf_conn_il, Description: eDocument Inter

In [4]:
CODE_SEARCHER_SETTINGS = CodebaseSearcherSettings()

MEMORY_MANAGER = MemoryManager(
    memory_store_type="Postgres",
    namespace=CODE_SEARCHER_SETTINGS.namespace,
    embedding_fields=["description"],
)

In [ ]:
code_lookup = SourceCodeRetriever()

for my_class in classes:
    print("Class name: ", my_class[0])
    class_name = my_class[0]
    class_description = my_class[1]

    class_code = code_lookup._run(class_name=class_name)

    print("\t- Started indexing")
    repository = ABAPClassRepository(source_code=class_code)
    print(
        f"\t- Finished indexing, number of methods: {len(repository.classes.get(class_name).keys())}"
    )

    for method_name in repository.classes[class_name].keys():
        if not MEMORY_MANAGER.is_memory_present(f"{class_name}.{method_name}"):
            print(f"\t- Summarizing and adding method: {method_name}")
            # Summarize the code of the method and add it to the memory
            memory_content = {
                "description": CodeSummarizer.summarize_code(
                    f"Class description: {class_description}\n"
                    + repository.get_content_by_class_and_method(
                        class_name=class_name, method_name=method_name
                    )
                ),
                "code": repository.get_content_by_class_and_method(
                    class_name=class_name, method_name=method_name
                ),
            }
            MEMORY_MANAGER.add_memory(
                memory_title=f"{class_name}.{method_name}",
                memory_content=memory_content,
            )
        else:
            print(f"\t- Skipping method: {method_name}")
            continue

Class name:  cl_edoc_sg_apj_jt_check
	- Started indexing
	- Finished indexing, number of methods: 8
	- Skipping method: if_apj_jt_check_20~adjust_hidden
	- Skipping method: if_apj_jt_check_20~adjust_read_only
	- Skipping method: if_apj_jt_check_20~check_and_adjust
	- Skipping method: if_apj_jt_check_20~check_and_adjust_parameter
	- Skipping method: if_apj_jt_check_20~check_authorizations
	- Skipping method: if_apj_jt_check_20~check_start_condition
	- Skipping method: if_apj_jt_check_20~get_dynamic_properties
	- Skipping method: if_apj_jt_check_20~initialize
Class name:  cl_edoc_map_escan_fica
	- Started indexing
	- Finished indexing, number of methods: 12
	- Skipping method: agg_re_out_invtax_fica
	- Skipping method: agg_re_out_invtax_fica_wo_eq
	- Skipping method: class_constructor
	- Skipping method: fill_header_can
	- Skipping method: get_mapping_badi_fica
	- Skipping method: map_de_ou_inv1
	- Skipping method: map_de_ou_invr1
	- Skipping method: map_re_out_inv1_fica_tax
	- Skipping 

KeyboardInterrupt: 